<a href="https://colab.research.google.com/github/kobrue02/BENALI/blob/main/notebook/llama_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()

In [23]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")

def get_l1_prediction(text, l1s):
    prompt = """
    Consider the following text which was written by a non-native speaker: \n
    `{text}`
    \n\n What do you think is their native language?
    Choose on of the following:
    {labels}
    Return only the label.
    """.format(text=text, labels=l1s)
    messages = [
        {"role": "user", "content": prompt},
    ]
    return pipe(messages)[0]["generated_text"][1]["content"]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
import pandas as pd

test_df = pd.read_csv("nli_test.csv")
test_df.head()

,sentence,l1
0,It would be moree accurate to say Athens (3 mi...,Greece
1,#编号：540\n#篇名：兄弟射雁\n#字数／字符数：411/411\n#类型：习作\n#姓...,Vietnam
2,"Not that they are poor, they just don't give m...",Portugal
3,Red in the corners with white surrounding them,Norway
4,>Wenn es stets zu Schutz und Trutze,Germany


In [5]:
labels = "\n".join(test_df["l1"].unique())
labels

'Greece\nVietnam\nPortugal\nNorway\nGermany\nSlovenia\nRussia\nSweden\nJapan\nFinland\nIreland\nBelgium\nTurkey\nDenmark\nBosnia and Herzegovina\nItaly\nPoland\nCzechia\nRomania\nNorth Macedonia\nUnited Kingdom\nNetherlands\nHungary\nFrance\nIceland\nMontenegro\nSerbia\nBulgaria\nUnited States\nArgentina\nAustria\nSwitzerland\nKazakhstan\nGreenland\nSpain\nMadagascar\nCanada\nEngland\nCroatia\nUkraine\nLithuania\nIndonesia\nCyprus\nIsrael\nWales\nChina\nScotland\nEstonia\nBelarus\nSlovakia\nIran\nSouth Korea\nLatvia\nMoldova\nLuxembourg\nBrazil\nSaudi Arabia\nAlbania\nColombia\nAustralia\nPalestine\nNew Zealand\nBavaria\nAzerbaijan\nThailand\nNigeria\nArmenia\nUzbekistan\nMyanmar\nPhilippines\nIndia\nMalta\nAndorra\nMexico\nKyrgyzstan\nMongolia\nAstrakhan\nLaos\nChile\nTagalog\nCambodia\nGeorgia\nSaint Helena\nVatican City\nMalaysia\nOirat\nTajikistan\nPeru\nPanama\nRepublic of the Congo'

In [6]:
test_sample = test_df.iloc[0]
test_sample

,0
sentence,It would be moree accurate to say Athens (3 mi...
l1,Greece


In [10]:
model_response = get_l1_prediction(test_sample["sentence"], labels)
model_response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Bosnia and Herzegovina'

In [ ]:
# on the full test set
from tqdm import tqdm
from transformers import logging as transformers_logging
import warnings

transformers_logging.set_verbosity_error()
warnings.filterwarnings("ignore")
tqdm.pandas()

test_df["l1_pred"] = test_df.progress_apply(lambda x: get_l1_prediction(x["sentence"], labels), axis=1)

  0%|          | 545/161994 [02:44<19:49:24,  2.26it/s]

In [22]:
import torch
torch.cuda.empty_cache()

# also run garbage collection
import gc
gc.collect()

7883